In [ ]:
import numpy as np
import sklearn.metrics.pairwise as pairwise

In [4]:
x=np.loadtxt('/home/Documents/deeprepresentation/wine.txt',dtype='float',delimiter=',',usecols = list(range(1,14)))

IOError: [Errno 2] No such file or directory: '/home/ankita/Desktop/data/wine.txt'

In [ ]:
x

NameError: name 'x' is not defined

In [ ]:
labels=np.loadtxt('/home/Documents/deeprepresentation/wine.txt',dtype='float',delimiter=',',usecols = (0,))

IOError: [Errno 2] No such file or directory: '/home/ankita/Desktop/data/wine.txt'

In [60]:
labels
labels.shape

(178,)

In [61]:
v = labels.transpose()
v.shape

(178,)

In [6]:
x

array([[  1.42300000e+01,   1.71000000e+00,   2.43000000e+00, ...,
          1.04000000e+00,   3.92000000e+00,   1.06500000e+03],
       [  1.32000000e+01,   1.78000000e+00,   2.14000000e+00, ...,
          1.05000000e+00,   3.40000000e+00,   1.05000000e+03],
       [  1.31600000e+01,   2.36000000e+00,   2.67000000e+00, ...,
          1.03000000e+00,   3.17000000e+00,   1.18500000e+03],
       ..., 
       [  1.32700000e+01,   4.28000000e+00,   2.26000000e+00, ...,
          5.90000000e-01,   1.56000000e+00,   8.35000000e+02],
       [  1.31700000e+01,   2.59000000e+00,   2.37000000e+00, ...,
          6.00000000e-01,   1.62000000e+00,   8.40000000e+02],
       [  1.41300000e+01,   4.10000000e+00,   2.74000000e+00, ...,
          6.10000000e-01,   1.60000000e+00,   5.60000000e+02]])

In [210]:
xtrain = pairwise.rbf_kernel(x,x,gamma=1.0/(2*490))

In [211]:
xtrain

array([[  1.00000000e+000,   3.68818731e-001,   2.05997788e-007, ...,
          3.22111554e-024,   3.32822007e-023,   3.22898006e-114],
       [  3.68818731e-001,   1.00000000e+000,   7.88033430e-009, ...,
          1.91276729e-021,   1.69912115e-020,   3.13077399e-107],
       [  2.05997788e-007,   7.88033430e-009,   1.00000000e+000, ...,
          3.43444668e-055,   1.20496736e-053,   7.12314865e-174],
       ..., 
       [  3.22111554e-024,   1.91276729e-021,   3.43444668e-055, ...,
          1.00000000e+000,   9.71121417e-001,   1.66333393e-034],
       [  3.32822007e-023,   1.69912115e-020,   1.20496736e-053, ...,
          9.71121417e-001,   1.00000000e+000,   9.78689720e-036],
       [  3.22898006e-114,   3.13077399e-107,   7.12314865e-174, ...,
          1.66333393e-034,   9.78689720e-036,   1.00000000e+000]])

In [212]:
D = np.diag(1.0/np.sqrt(xtrain.sum(axis= 1)))
nxtrain = D.dot(xtrain).dot(D)
#nxtrain = xtrain/xtrain.sum(axis= 1)
print nxtrain

[[  1.77748824e-001   5.17984564e-002   4.79817109e-008 ...,
    5.45801736e-025   5.50348730e-024   4.03125365e-115]
 [  5.17984564e-002   1.10968818e-001   1.45029025e-009 ...,
    2.56087191e-022   2.21997146e-021   3.08832923e-108]
 [  4.79817109e-008   1.45029025e-009   3.05224654e-001 ...,
    7.62591261e-056   2.61100470e-054   1.16534149e-174]
 ..., 
 [  5.45801736e-025   2.56087191e-022   7.62591261e-056 ...,
    1.61528992e-001   1.53081017e-001   1.97959378e-035]
 [  5.50348730e-024   2.21997146e-021   2.61100470e-054 ...,
    1.53081017e-001   1.53831448e-001   1.13668201e-036]
 [  4.03125365e-115   3.08832923e-108   1.16534149e-174 ...,
    1.97959378e-035   1.13668201e-036   8.76885297e-002]]


In [204]:
np.linalg.norm(nxtrain)/178

0.013745368409439507

In [54]:
import keras
from keras.layers import Input, Dense, Activation, Dropout,Activation
from keras.optimizers import Adam, Nadam, RMSprop, Adadelta
from keras import regularizers 
from keras.models import Model

In [40]:
def LeakyReLU(alpha):
    return Activation('sigmoid')

In [62]:
inputs = Input(shape=(178,))

model = Dense(128)(inputs)
#model = LeakyReLU(alpha=0.1)(model)
model = sigmoid(alpha = 0.1)(model)
model = Dropout(0.8)(model)

model = Dense(64, activity_regularizer = regularizers.activity_l1(10e-5),name='embed',activation='sigmoid')(model)

model = Dropout(0.8)(model)

model = Dense(128)(model)
model = LeakyReLU(alpha=0.1)(model)
model = Dropout(0.8)(model)

model = Dense(178)(model)
model = LeakyReLU(alpha=0.1)(model)

opt=Adam(lr=0.005, decay=1e-2)
#opt=Adadelta()
ae = Model(input = inputs, output = model)
ae.compile(optimizer=opt, loss='mean_squared_error')
ae.fit(nxtrain,nxtrain,nb_epoch= 100, batch_size = 2, shuffle=True)

NameError: name 'Input' is not defined

In [95]:
ae.predict(np.expand_dims(nxtrain[0,:],-1)

(1, 178)

In [238]:
mm = Model(input= ae.input,output=ae.get_layer('embed').output)

In [239]:
mm.predict(np.expand_dims(nxtrain[0,:],0)).shape

(1, 64)

In [240]:
rec = mm.predict(nxtrain)

In [241]:
u,s,_ = np.linalg.svd(nxtrain)

In [248]:
orig= u[:,:3]

In [249]:
np.linalg.norm(orig-rec)

ValueError: operands could not be broadcast together with shapes (178,3) (178,64) 

In [244]:
(1)

1

In [250]:
from sklearn.cluster import KMeans
from sklearn import metrics

kmeans_bare = KMeans(n_clusters=3,init='random', random_state=None,max_iter=500).fit(x)
kmeans_rec = KMeans(n_clusters=3, random_state=0).fit(rec)
kmeans_orig = KMeans(n_clusters=3, random_state=0).fit(orig)
kmeans_rec.labels_,kmeans_orig.labels_
ss_rec = metrics.silhouette_score(nxtrain, kmeans_rec.labels_, metric='euclidean', sample_size=None,random_state=None)
ss_orig = metrics.silhouette_score(nxtrain, kmeans_orig.labels_, metric='euclidean', sample_size=None,random_state=None)
ss_bare = metrics.silhouette_score(x, kmeans_bare.labels_, metric='euclidean', sample_size=None,random_state=None)

ri_rec = metrics.adjusted_rand_score(kmeans_rec.labels_,labels)
ri_orig = metrics.adjusted_rand_score(kmeans_orig.labels_,labels)
ri_bare = metrics.adjusted_rand_score(kmeans_bare.labels_,labels)

nmi_rec = metrics.normalized_mutual_info_score(kmeans_rec.labels_,labels)
nmi_orig = metrics.normalized_mutual_info_score(kmeans_orig.labels_,labels)
nmi_bare = metrics.normalized_mutual_info_score(kmeans_bare.labels_,labels)
print kmeans_bare.labels_
ss_rec,ss_orig, ri_rec, ri_orig,ri_bare, nmi_rec,nmi_orig,nmi_bare

[2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 2 2 0 0 2 2 0 2 2 2 2 2 2 0 0
 2 2 0 0 2 2 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 0 1 0 1 1 0 1 1 0 0 0 1 1 2
 0 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 1 0 1 1 1 0 1 1 1 1 0 1
 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0
 0 1 1 1 1 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1]


(0.051664347075296151,
 0.4127925661447272,
 0.051398038976261912,
 0.0054438354437086308,
 0.37111371823084754,
 0.088370527034240726,
 0.09143515133814191,
 0.42875686335053037)

In [246]:

def randindex(labels1,labels2):
    tp,tn,fp,fn = 0.0,0.0,0.0,0.0
    for point1 in range(len(labels1)):
        for point2 in range(len(labels1)):
            tp += 1 if labels1[point1] == labels1[point2] and labels2[point1] == labels2[point2] else 0
            tn += 1 if labels1[point1] != labels1[point2] and labels2[point1] != labels2[point2] else 0
            fp += 1 if labels1[point1] != labels1[point2] and labels2[point1] == labels2[point2] else 0
            fn += 1 if labels1[point1] == labels1[point2] and labels2[point1] != labels2[point2] else 0
    return (tp+tn) /(tp+tn+fp+fn)

In [247]:
randindex(kmeans_rec.labels_,labels),randindex(kmeans_orig.labels_,labels),randindex(kmeans_bare.labels_,labels)

(0.5566847620249968, 0.3646635525817447, 0.720237343769726)

In [231]:
import numpy as np
import matplotlib.pyplot as plt
#from sklearn.clustering import SpectralClustering
from sklearn import cluster, datasets
from sklearn import metrics

numpy.linalg.eigh()
np.random.seed(0)
blobs = datasets.make_blobs(n_samples=1500, random_state=8)
X, y = blobs
X = StandardScaler().fit_transform(X)
spectral = cluster.SpectralClustering(n_clusters=3,eigen_solver='arpack')
#spectral = cluster.SpectralClustering(n_clusters=3,eigen_solver='arpack',affinity = 'nearest_neighbors')
spectral.fit(X)
if hasattr(spectral, 'labels_'):
        y_pred = spectral.labels_.astype(np.int)
else:
        y_pred = spectral.predict(X)

ss = metrics.silhouette_score(X, y_pred, metric='euclidean', sample_size=None,random_state=None)

print(ss)

NameError: name 'numpy' is not defined

In [79]:
dir(ae)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collected_trainable_weights',
 '_fit_loop',
 '_function_kwargs',
 '_get_node_attribute_at_index',
 '_make_predict_function',
 '_make_test_function',
 '_make_train_function',
 '_output_mask_cache',
 '_output_shape_cache',
 '_output_tensor_cache',
 '_predict_loop',
 '_standardize_user_data',
 '_test_loop',
 '_updated_config',
 'add_inbound_node',
 'add_loss',
 'add_update',
 'add_weight',
 'assert_input_compatibility',
 'build',
 'built',
 'call',
 'compile',
 'compute_mask',
 'constraints',
 'container_nodes',
 'count_params',
 'create_input_layer',
 'evaluate',
 'evaluate_generator',
 'fit',
 'fit_generator',
 'from_config',
 'get_config',
 'get_input_at',
 'get_input_mask_at',
 'get_input_shape_at',
 'ge

In [85]:
ae.get_layer('embed').output

<tf.Tensor 'add_1468:0' shape=(?, 64) dtype=float32>